# Delta Airlines Twitter Customer Support Analysis

In this notebook we will be analyzing tweets from users for Delta Air Lines support reps using OpenAI GPT3.5 API. The analysis is done in 2 steps:

1. Intent identification: find out why a user is writing a tweet, this could be one of 4 reasons:
    - Raise a complaint/grievance
    - Ask a question
    - Share a good experience
    - Other reasons

2. Reasons for complaints: for the tweets where a complaint is being raised, we want to identify the major causes which would be one of:
    - Flight Cancellation
    - Flight Delays
    - Bad Flight Experience
    - Lost/Damaged Luggage
    - Flight Attendant Complaints
    - Flight Booking Problems
    - Poor Customer Service
    - Other

### Data Source

The data used here is a subset of the [Customer Support on Twitter](https://www.kaggle.com/datasets/thoughtvector/customer-support-on-twitter) dataset on Kaggle. The original dataset consists of both tweets and replies for many customer support channels.

In this analysis, we will be using only the inbound tweets for Delta Air Lines. This data has been exported to the `data` dir.

### Environment Setup
This notebook was run using Python 3.8 with following packages installed:
- langchain==0.0.177
- pandas==2.0.1

## Load data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./data/delta-airlines-tweets.csv")
df.shape

(1066, 8)

## Setup langchain

- [OpenAI Chat API reference](https://platform.openai.com/docs/guides/chat/introduction)
- [Langchain API reference](https://python.langchain.com/en/latest/modules/models/chat/integrations/openai.html)

Environment file:
```
OPENAI_API_KEY=<your-open-ai-key>
```

In [3]:
# load open ai api key
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

In [5]:
def build_chat_prompt(template):
    system_message_prompt = SystemMessagePromptTemplate.from_template("")
    user_message_prompt = HumanMessagePromptTemplate.from_template(template)
    return ChatPromptTemplate.from_messages([
        system_message_prompt,
        user_message_prompt])

In [6]:
# temperature=0 makes the results a bit more deterministic
chat = ChatOpenAI(temperature=0)

## Get tweet category

In [7]:
df.head(2)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,date
0,586413,165165,True,Sun Dec 03 15:22:01 +0000 2017,@Delta I do really appreciate the app notifica...,586412,NaN,2017-12-03
1,581584,243849,True,Sun Dec 03 12:03:39 +0000 2017,TYSM @Delta #472 for the fabulous flight #delt...,581583,NaN,2017-12-03


In [8]:
df.sample(10).text.tolist()

["I need my in-flight coffee @Delta, but I can't do black. Please add nondairy #vegan creamers. ☕️",
 "You know what's still a GREAT movie? Gladiator. \n\nThanks for making it an option @Delta.\n\nYes, I was entertained. https://t.co/bK2FD5jaHs",
 '50 mins on the runway @124202 awaited takeoff. After a 2 hour flight delay. What’s going on @delta????',
 '@delta love you guys..but I need to change my reward flight..need to come home two days earlier than expected. #lifeofanurse but system is down?',
 'I’m away from “home” about 260 days a year, some may say I travel a lot! First time EVER today flying @Delta, first leg great, lounge fantastic. One sector to go until I get “home”. So far I’m impressed!',
 '@Delta follow me please I enjoy your service',
 'Tasha was helpful. The other lady was rude. Because of that, the @delta flt w room for all of us filled up. #flyingsolonow #hopeweallmakeit #deltadoesntcare',
 'I got on my @Delta flight and asked for some water while we were still boardi

### setup prompt

In [9]:
category_prompt_template = """
you are a customer support agent for Delta Air Lines in the USA \
who is responding to tweets from users. given a tweet, you are supposed \
to classify it into one of the following categories:

- complaint
- question
- compliment

If a tweet does not fall into any of the above categories, mark it as "other".

classify the following tweet:

{tweet}
"""
chat_prompt = build_chat_prompt(category_prompt_template)

In [10]:
sample_tweet = df.sample(1).text.iloc[0]
sample_response = chat(chat_prompt.format_prompt(tweet=sample_tweet).to_messages())
print(f"tweet: {sample_tweet} \n\nresponse: {sample_response.content}")

tweet: @Delta in Canun and you cancelled my girlfriends flight for no reason thanks for nothing!!!
Rude staff, Horrible person ignoring me, unhelpful now I have to stay overnight in Atlanta terrible service.
I take several flights a year Delta are now bottom of my list. 

response: complaint


### run for entire dataset

In [11]:
batch_messages = [chat_prompt.format_prompt(tweet=tweet).to_messages()
                  for tweet in df.text]
len(batch_messages)

1066

Using the %time magic here to show the amount of time these API calls could take. This can be improved by reaching out to open ai support if you are putting code into production at scale.

In [12]:
%time result = chat.generate(batch_messages)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 615e0426bfa4e410c54e3cd74e9ad932 in your message.).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID cc296ac2f3c95aa5c8e2a394a94ff144 in your message.).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can r

CPU times: user 4 s, sys: 501 ms, total: 4.5 s
Wall time: 25min 14s


In [13]:
len(result.generations)

1066

In [14]:
df.loc[:, "category_raw"] = [x[0].text for x in result.generations]

In [15]:
df.category_raw.value_counts()

category_raw
complaint               424
compliment              333
question                246
other                    41
Category: Complaint       5
category: complaint       5
category: question        4
Question                  2
Category: Question.       2
Category: Complaint.      1
Category: question.       1
Category: question        1
suggestion                1
Name: count, dtype: int64

### parse output

In [16]:
import re

In [17]:
def extract_category(raw):
    pattern = re.compile("compliment|complaint|question|other")
    match = pattern.search(raw.lower())
    if match is None:
        return None
    return match.group()

In [18]:
df.loc[:, "category"] = df.category_raw.apply(extract_category)

In [19]:
df.loc[df.category.isnull()]

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,date,category_raw,category
928,511880,237698,True,Fri Dec 01 22:47:52 +0000 2017,"Hey @delta, ever considered offering #cancer p...",511879,NaN,2017-12-01,suggestion,None


In [20]:
df.category.value_counts()

category
complaint     435
compliment    333
question      256
other          41
Name: count, dtype: int64

In [21]:
def sample_category(category, n):
    print("\n\n".join(list(df.loc[df.category == category].text.sample(n))))

In [23]:
sample_category("question", 5)

@Delta my traveling companion was able to check in via mobile app but I can’t. Please help!

@Delta Why dont you include active service men &amp; women for priority boarding? I flew FT from SRQ-ATL &amp; noticed omission.
#HonorThoseWhoServeTheFlag

I need my in-flight coffee @Delta, but I can't do black. Please add nondairy #vegan creamers. ☕️

@delta what is the checked bag weight limit for domestic first class?  And restrictions on bag size?

@Delta traveling on dec 10th o delta to Amsterdam but my able to choose better seat please help.


In [24]:
df.to_csv("./data/delta-tweets-with-category.csv", index=False)

## Get reason for complaints

This is a good first start, but for an actual appication perspective we need to see where exactly we're going wrong. Let's try to classify the negative reviews further into some categories:

- Bad Flight Experience
- Flight Cancellation
- Flight Delays
- Poor Customer Service
- Damaged Luggage
- Flight Attendant Complaints
- Flight Booking Problems
- Lost Luggage
- Other

In [25]:
df_complaints = df.loc[df.category == "complaint"]
df_complaints.shape

(435, 10)

### setup prompt

In [26]:
neg_category_prompt_template = """
you are a customer support agent for one of the largest airlines in the USA \
who is responding to tweets from users. 

given a tweet where a customer is complaining, you are supposed to identify the \
reason for the complaint and classify it into one of the following categories:

- Flight Cancellation
- Flight Delays
- Bad Flight Experience
- Lost/Damaged Luggage
- Flight Attendant Complaints
- Flight Booking Problems
- Poor Customer Service

If a tweet does not fall into any of the above categories, mark it as "other".

classify the following tweet:

{tweet}
"""

In [27]:
chat_prompt_complaint = build_chat_prompt(neg_category_prompt_template)

In [30]:
sample_tweet = df_complaints.sample(1).text.iloc[0]
sample_response = chat(chat_prompt_complaint.format_prompt(tweet=sample_tweet).to_messages())
print(f"tweet: {sample_tweet} \n\nresponse: {sample_response.content}")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID b7816dcbe0e0ab68d70cc99eb97686e3 in your message.).


tweet: @Delta @824 33-55min wait time when I need an immediate response NOW since at airport?? wth Delta. 

response: Poor Customer Service.


### run at scale

In [31]:
batch_messages = [chat_prompt_complaint.format_prompt(tweet=tweet).to_messages()
                  for tweet in df_complaints.text]

In [32]:
%time result_complaint = chat.generate(batch_messages)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 21ab43c31eff05abbd2ecf316e289285 in your message.).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 078ad459241774b047cfee05ceff5907 in your message.).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can r

CPU times: user 1.69 s, sys: 218 ms, total: 1.9 s
Wall time: 14min 15s


In [34]:
df_complaints.loc[:, "complaint_category_raw"] = [x[0].text for x in result_complaint.generations]

In [35]:
df_complaints.complaint_category_raw.value_counts()

complaint_category_raw
Flight Booking Problems                                                                                                                                                                                              86
Category: Flight Delays                                                                                                                                                                                              62
Category: Bad Flight Experience.                                                                                                                                                                                     53
Lost/Damaged Luggage.                                                                                                                                                                                                30
Category: Poor Customer Service                                                                                  

### parse output

In [38]:
neg_categories = [
    "Flight Cancellation",
    "Flight Delays",
    "Bad Flight Experience",
    "Lost/Damaged Luggage",
    "Flight Attendant Complaints",
    "Flight Booking Problems",
    "Poor Customer Service",
    "Other"
]
def extract_complaint_category(raw):
    pattern = re.compile("|".join(neg_categories), re.IGNORECASE)
    match = pattern.search(raw.lower())
    if match is None:
        return None
    return match.group()

In [39]:
df_complaints.loc[:, "category"] = df_complaints.complaint_category_raw.apply(
    extract_complaint_category)

In [40]:
df_complaints.category.value_counts()

category
flight booking problems        118
flight delays                   88
bad flight experience           78
poor customer service           62
lost/damaged luggage            43
other                           17
flight attendant complaints     14
flight cancellation             10
Name: count, dtype: int64

In [41]:
df_complaints.loc[df_complaints.category.isnull()]

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,date,category_raw,category,complaint_category_raw
41,586419,258486,True,Sun Dec 03 15:13:56 +0000 2017,@DELTA so is policy to install navigation upgr...,"586418,586420",NaN,2017-12-03,complaint,None,Flight Experience Complaint.
96,565474,247709,True,Sun Dec 03 00:40:34 +0000 2017,This is the 2nd flight I’ve been on today @Del...,565472,NaN,2017-12-03,complaint,None,Flight Experience - WiFi not working on multip...
262,597387,183666,True,Sun Dec 03 19:51:39 +0000 2017,@2838 why do you only have one bus on Friday n...,597385,NaN,2017-12-03,complaint,None,Flight Experience - Inconvenient airport trans...
356,535349,244033,True,Sat Dec 02 12:22:39 +0000 2017,@delta who do I submit my complaint to regardi...,535348,NaN,2017-12-02,Category: Complaint,None,Flight Experience Complaint.
1026,504411,235616,True,Fri Dec 01 19:42:50 +0000 2017,First world problem. @delta is leaving Marine ...,504410,NaN,2017-12-01,complaint,None,Category: Flight Experience Complaints.


In [42]:
df_complaints.to_csv("./data/delta-tweets-complaint-category.csv")

## What did this cost us?

As per the [pricing](https://openai.com/pricing#language-models) documentation of Open AI, the gpt-3.5-turbo api is charged at \\$0.001 for every 1K tokens. Below we can see that we used 206677 tokens in total, resulting in a total cost of **\\$0.41**. This is fairly low, but we only passed in a 1000 tweets. The cost would ramp up as we analyze more data.

In [44]:
all_results = [result, result_complaint]
total_tokens = sum([response.llm_output["token_usage"]["total_tokens"] for response in all_results])
cost = total_tokens / 1000 * 0.002
print(f"Tokens used: {total_tokens} | Totak cost: ${cost}")

Tokens used: 206677 | Totak cost: $0.413354
